In [1]:
import sys
import glob
import errno

from astropy.io import ascii
from astropy.table import Table, Column, join
from astropy.io import fits
import numpy as np

#Function for easy column manipulation
def column(matrix, i):
    return[row[i] for row in matrix]

#input and output files
path = '../data/TeVSpectra/SpectraBW15/*.ecsv'
files = glob.glob(path) 
rs=ascii.read('../data/TeVSpectra/SourcesBW15_v2.ecsv')

outfilename = '../data/CompiledTeVSources.fits'

In [2]:
#initialize
f, cols, n, a, b, c, d = ([] for i in range(7))
s, r, ra, dec, safe, fgl = ([] for i in range(6))

#get redshift info
s.append(column(rs,'SOURCE'))
r.append(column(rs,'REDSHIFT'))
ra.append(column(rs,'RA_J2000'))
dec.append(column(rs,'DEC_J2000'))
safe.append(column(rs,'IS_REDSHIFT_SAFE'))
fgl.append(column(rs,'3FGL_NAME'))


In [3]:
#loop over files
for i, name in enumerate(files):
    f.append(ascii.read(name))
    #get name for file
    n.append(name[31:-5])
    
    #make all the columns the same length (fill extra slots with nan's)
    for j in range(25-len(f[i])):
        f[i].add_row([np.nan,np.nan,np.nan,np.nan])

    a.append(column(f[i],'E_REF'))
    b.append(column(f[i],'NORM'))
    c.append(column(f[i],'NORM_ERRP'))
    d.append(column(f[i],'NORM_ERRN'))


In [4]:
sf = ["" for x in range(len(n))]
fglf=["" for y in range(len(n))]
safef = [False for x in range(len(n))]
rf, raf, decf = (np.full(len(n),np.nan) for i in range(3))

for k in range(len(s[0])):
    for kk in range(len(n)):
        if s[0][k] in n[kk]:
            #print s[0][k], n[kk]
            sf[kk]=s[0][k] 
            rf[kk]=r[0][k]
            raf[kk]=ra[0][k] 
            decf[kk]=dec[0][k] 
            safef[kk]=safe[0][k] 
            fglf[kk]=fgl[0][k]


In [5]:
#Building columns for table
cols =  [Column(name='3FGL_NAME',dtype='S32',data=fglf)]
cols += [Column(name='SOURCE_FULL',dtype='S32',data=n)]
cols += [Column(name='SOURCE',dtype='S32',data=sf)]
cols += [Column(name='REDSHIFT',dtype='f8',data=rf)]
cols += [Column(name='RA_J2000',dtype='f8',data=raf)]
cols += [Column(name='DEC_J2000',dtype='f8',data=decf)]
cols += [Column(name='IS_REDSHIFT_SAFE',dtype='bool',data=safef)]
cols += [Column(name='E_REF',dtype='f8',data=a)]
cols += [Column(name='NORM',dtype='f8',data=b)]
cols += [Column(name='NORM_ERRP',dtype='f8',data=c)]
cols += [Column(name='NORM_ERRN',dtype='f8',data=d)]
t = Table(cols)

In [6]:
print t
t.write(outfilename,format='fits',overwrite=True)

 3FGL_NAME             SOURCE_FULL            ...   NORM_ERRN [25]  
------------ -------------------------------- ... ------------------
J0232.8+2016       1ES0229+200_HESS_2005-2006 ...  1.3277e-08 .. nan
J0232.8+2016    1ES0229+200_VERITAS_2009-2012 ...     2.8e-08 .. nan
J0349.2-1158      1ES0347-121_HESS_2006-09-12 ...  3.5758e-08 .. nan
J0416.8+0104       1ES0414+009_HESS_2005-2009 ...   1.017e-07 .. nan
J0416.8+0104    1ES0414+009_VERITAS_2008-2011 ...    1.09e-08 .. nan
J0809.8+5218    1ES0806+524_VERITAS_2006-2008 ... 9.29454e-08 .. nan
J1015.0+4925           1ES1011+496_MAGIC_2007 ...    1.69e-06 .. nan
J1103.5-2329       1ES1101-232_HESS_2004-2005 ...  1.0851e-07 .. nan
J1217.8+3007           1ES1215+303_MAGIC_2011 ...    1.53e-06 .. nan
J1217.8+3007    1ES1215+303_VERITAS_2008-2012 ...  5.7133e-08 .. nan
         ...                              ... ...                ...
J2158.8-3013      PKS2155-304_HESS_2006-08-03 ...   2.947e-07 .. nan
J2158.8-3013      PKS2155-304_HESS